In [2]:
pip install graphviz ipywidgets joblib


  Using cached graphviz-0.21-py3-none-any.whl.metadata (12 kB)
  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached widgetsnbextension-4.0.14-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl.metadata (20 kB)
Using cached graphviz-0.21-py3-none-any.whl (47 kB)
Using cached ipywidgets-8.1.7-py3-none-any.whl (139 kB)
Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl (216 kB)
Using cached widgetsnbextension-4.0.14-py3-none-any.whl (2.2 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import joblib
from sklearn.tree import export_graphviz
import graphviz

# Load model
rf = joblib.load("output/models/RandomForest_production.pkl")

# Feature names from your dataset
feature_names = [
    "AnomalyScore","RegionArea","MaxVal","RegionCount",
    "Burn Through","Good","Notching","Porosity",
    "CL_ConfMax","CL_ConfMargin","Row1","Col1",
    "Length1","Length2","PHI","OD1_ClassID","OD1_Conf"
]

# Export tree #0
dot_data = export_graphviz(
    rf.estimators_[0],
    out_file=None,
    feature_names=feature_names,
    class_names=rf.classes_.astype(str),
    filled=True,
    rounded=True,
    special_characters=True
)

# Render to PDF and open it
graph = graphviz.Source(dot_data)
graph.render("rf_tree_0", format="pdf", view=True)

print("✅ Tree rendered to rf_tree_0.pdf")


✅ Tree rendered to rf_tree_0.pdf


In [4]:
import joblib
import xgboost as xgb
from xgboost import to_graphviz
import graphviz
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load XGBoost model
model = joblib.load("output/models/XGBoost_production.pkl")

# Feature names from your dataset
feature_names = [
    "AnomalyScore","RegionArea","MaxVal","RegionCount",
    "Burn Through","Good","Notching","Porosity",
    "CL_ConfMax","CL_ConfMargin","Row1","Col1",
    "Length1","Length2","PHI","OD1_ClassID","OD1_Conf"
]

# Total number of boosting rounds (trees)
num_trees = model.get_booster().num_boosted_rounds()

# Slider to choose tree
tree_slider = widgets.IntSlider(0, 0, num_trees - 1, step=1, description="Tree #")
export_btn = widgets.Button(description="Export PDF")

def show_tree(tree_index):
    clear_output(wait=True)
    display(tree_slider, export_btn)

    g = to_graphviz(model, num_trees=tree_index, rankdir="TB", 
                    fmap="",  # use default feature map
                    num_features=len(feature_names),
                    yes_color='#0000FF', no_color='#FF0000')
    
    display(g)

def on_slider_change(change):
    show_tree(change["new"])

def on_export(_):
    idx = tree_slider.value
    g = to_graphviz(model, num_trees=idx, rankdir="TB")
    filename = f"xgb_tree_{idx}"
    g.render(filename, format="pdf", view=True)
    print(f"✅ Exported {filename}.pdf")

tree_slider.observe(on_slider_change, names="value")
export_btn.on_click(on_export)

# Show first tree
show_tree(0)


IntSlider(value=0, description='Tree #', max=399)

Button(description='Export PDF', style=ButtonStyle())

c:\Users\QuinnMazaris\Desktop\ensamble_pipelineV2\venv\Lib\site-packages\xgboost\plotting.py:267: FutureWarning: The `num_trees` parameter is deprecated, use `tree_idx` insetad. 
  warnings.warn(


XGBoostError: [08:32:38] C:\actions-runner\_work\xgboost\xgboost\src\tree\tree_model.cc:594: Failed to parse graphviz parameters:
	{"graph_attrs": {"rankdir": "TB", "num_features": 17}, "edge": {"yes_color": "#0000FF", "no_color": "#FF0000"}}
With error:
[08:32:38] C:\actions-runner\_work\xgboost\xgboost\dmlc-core\include\dmlc/json.h:677: Check failed: ch == '\"' (49 vs. ") : Error at Line 0, around ^`7}, "edge": {"yes_color": "#0000FF", "no_color": "#FF0000"}}`, Expect '"' but get '1'